# Trabajo de las N - Reinas

# **00 JERARQUÍA Y ESTRUCTURA DEL PROYECTO**

Este proyecto está organizado modularmente para permitir la **comparación directa de rendimiento** entre las diferentes formas de abordar el problema. La arquitectura, se divide en tres niveles de control:

### 1. Nivel de Control: El Script Principal

El archivo **`principal`** (o el *notebook* que lo contiene) actúa como la unidad central de comando. Es el encargado de:

* Definir el parámetro $N$ (tamaño del tablero).
* Llamar, de forma secuencial, a los tres algoritmos de solución.
* Recopilar y almacenar los **tiempos de ejecución**.
* Coordinar el flujo de datos hacia la visualización final.

---

### 2. Nivel de Lógica: Algoritmos de Solución (Benchmarking)

El corazón del proyecto son los tres códigos de búsqueda. Se diferencian por su enfoque en la **velocidad** versus la **garantía matemática**.

| Método | Paradigma / Enfoque | Objetivo Clave |
| :--- | :--- | :--- |
| **Backtracking** | **Completo (Garantía)** | Encontrar **todas** las soluciones posibles. |
| **Manipulación de Bits** | **Optimizado (Velocidad)** | Ejecutar el Backtracking de la forma más **rápida** posible. |
| **Min-Conflicts** | **Heurístico (Escalabilidad)** | Encontrar **una** solución rápidamente en tableros muy grandes ($N \gg 100$). |

---

### 3. Nivel de Presentación: Recursos Externos

* **Librería `matplotlib`:** Se utiliza para generar la **visualización** de la solución (el tablero con las reinas `♛`) y para trazar las **gráficas de rendimiento** que comparan los tiempos de los tres métodos implementados.
* **Archivo `README`:** Es la **carta de presentación** del repositorio. Documenta el proyecto, lista los requisitos (`numpy`, `matplotlib`, etc.) y explica cómo ejecutar el código. El archivo REAMDME es de gran importancia, sobre todo si los autores del proyecto pretenden compartir su proyecto un repositorio, para facilitar el entendimiento de los lectores ajenos.

# **01 ORIGEN Y DEFINICIÓN**

El problema de las **N-Reinas** tiene una historia que se remonta al ajedrez, pero rápidamente se convirtió en un pilar de la informática.

* **Origen:** El desafío original fue planteado en **1848** por el ajedrecista alemán **Max Bezzel**, que se centró en el tablero estándar de **8x8**.
* **Generalización:** Dos años después, el matemático **Franz Nauck** amplió el problema a cualquier tablero de $N \times N$, dando lugar al problema que conocemos hoy.

**La Regla de Oro:**
La meta es sencilla: colocar $N$ reinas en un tablero $N \times N$ de manera que **ninguna de ellas se ataque**. Esto impone tres restricciones fundamentales:

1.  No deben compartir la misma **fila**.
2.  No deben compartir la misma **columna**.
3.  No deben compartir ninguna **diagonal**.

**Contexto del Código:**
El enfoque del problema se convierte en una búsqueda sistemática de un vector de posiciones $\vec{v} = [c_0, c_1, \dots, c_{N-1}]$, donde $c_i$ es la columna de la reina colocada en la fila que se corresponde con el subíndice $i$ de dicho vector, y debe satisfacer la restricción de las diagonales: $|c_i - c_j| \neq |i - j|$ para cualquier par $i \neq j$. De esta manera, la solución o soluciones múltiples de problema se simplificarán, describiendo el tablero con un arreglo unidimensional.

Basándonos en las 3 **Reglas de Oro**, planteamos las condiciones que se han de cumplir en nuestro código:


[texto del enlace](https://)


# **02 BACKTRACKING CLÁSICO**
 El backtracking es una técnica de programación basada en una busqueda sistemática a través de todos los caminos posibles.Para conseguir esto, por medio del backtracking se generan posibles soluciones candidatas que se van construyendo paso a paso.Si en algún momento alguna de estas posibles soluciones incumpliese las restricciones del problema(como que haya dos reinas en la misma fila,columna o diagonal),dicha solución se descarta y se vuelve hacia atrás para comprobar otro camino distinto.Así sucesivamente hasta que se hallan las soluciones que satisfagan las condiciones del problema.


 ### Explicación del código a partir de backtracking
Para simplificar el problema, el tablero(que es la variable que guarda las posiciones de las reinas en el tablero nxn) es una matriz nxn "camuflada" como lista la cual funciona de manera que cada índice de la lista corresponde a una fila del tablero y el valor albergado en cada posición, indica la columna donde se ha colocado la reina en esa fila.

Por otro lado,para la realización de este código se han usado 3 funciones cada una con una distinta funcionalidad.En primer lugar, la primera función es el cuerpo de nuestro código la cual nos permitirá implementar el backtracking posteriormente.Esta primera función es la que se va a encargar de verificar si el lugar donde se coloca la reina en cada posición acata las restricciones del problema(que no haya dos reinas en una misma fila,columna o diagonal).

1.La primera condición verifica que no estén en la misma columna.Para que dos reinas estén en la misma columna se tiene que cumplir que:
###                                           ∣Δcolumna∣=0


2.La segunda condición verifica que no estén en una misma diagonal.Para que dos reinas estén en la misma diagonal se tiene que cumplir que:
###                                          |Δfila| == |Δcolumna|


In [ ]:
def is_valid(row, col, queens):
    """Verifica si es seguro colocar una reina en la posición (row, col)."""
    #Creamos un bucle que recorra las filas de todo el tablero,por lo que no es necesario comprobar si están en la misma fila.
    #De esta manera, aparecen variables r y queens[r] que se refiere a las filas y columnas donde ya hay una reina.
    for r in range(row):
         #Para comprobar si están en la misma columna
        if col - queens[r]==0:
            return False
        #Para comprobar si estan en la misma diagonal(ponemos valor absoluto porque si ponemos la diagonal izquierda en uno sale 1 y otro -1)
        elif abs(col - queens[r]) == abs(row - r):
            return False
    #Se coloca la reina ya que es seguro
    return True


Para la realización de backtracking se definen dos funciones.Por un lado, una primera función que es la que establece el el tamaño n de tablero que necesitamos y el número de soluciones que queremos obtener.Dentro de esta, se define la variable "queens" que es una lista temporal que se va modificando de manera que se sustituyen sus valores por otros numeros que equivalen a la columna donde esta la reina,todo esto hasta que se da con una solución.Asímismo, cuando se da con una solución, se guarda dentro de la lista "solutions"(que va guardando todas las soluciones del problema).


Por otro lado, se define una segunda función(dentro de esta última) que recorre todas las columnas de cada fila para ver dónde se puede colocar una reina, utilizando la función "is_valid"(definida al principio del código). Si nos devuelve un True ,es decir, si la posición es válida , se coloca la reina en la columna de la fila dada y se llama a la funcion "place_queens" de forma recursiva para avanzar a la siguiente fila,de es. La recursión termina cuando se encuentra una solución completa(entonces dicha solución se guarda en la lista "solutions") o cuando no se puede seguir avanzando,en ambos casos se retrocede(backtracking) con queens[row] = -1 para probar otras columnas y seguir buscando todas las posibles soluciones.
Cabe recalcar que cuando se guarda una solución,toda la lista "queens" vuelve a estar llena de -1 gracias a la llamada recursiva explicada anteriormente.


In [ ]:

def solve_n_queens_multiple(n, limite_soluciones=5):

    #Encuentra hasta 'limite_soluciones' distintas para N reinas usando Backtracking.

    queens = [-1] * n
    solutions = []
    contador_iteraciones=0
    def place_queen(row):

        # Si ya tenemos suficientes soluciones, paramos la búsqueda para no saturar
        if len(solutions) >= limite_soluciones:
            return

        if row == n:
            #Si las filas==n eso significa que el programa ya ha colocado todas las reinas posibles(porque ya no existen más filas )
            # por lo que ha encontrado una solución,entonces la imprimimos
            solutions.append(list(queens))
            #este return detiene la recursión ya que una solución ha sido encontrada
            return
        #Recorremos todas las columnas posibles
        for col in range(n):
            contador_iteraciones+=1
            #Utilizamos la primera función que nos indica si es seguro colocar una reina en esa posición
            if is_valid(row, col, queens):
                #Si es seguro entonces coloca una reina en la fila y columnas dadas
                queens[row] = col
                #Ahora coloca la reina para la siguiente fila, hasta o bien encontrar la solución
                #o darse cuente de que no se puede con las colocaciones que se tenían anteriormente
                place_queen(row + 1)
                # Backtracking: quitamos la reina para probar la siguiente columna
                #el queens[row] = -1  solo se ejecuta cuando no se puede seguir avanzando con las reinas que se tenía

                queens[row] = -1

#Empieza colocando la reina en la fila 0

    place_queen(0)
    return solutions,contador_iteraciones
print(solve_n_queens_multiple(n=24))

### Código entero y funcionabilidad

In [ ]:
def is_valid(row, col, queens):
    for r in range(row):
        if col==queens[r]:
            return False
        elif abs(col - queens[r]) == abs(row - r):
            return False

    return True

def solve_n_queens_multiple(n, limite_soluciones=5):

    queens = [-1] * n
    solutions = []
    def place_queen(row):

        if len(solutions) >= limite_soluciones:
            return
        if row == n:
            solutions.append(list(queens))
            return
        for col in range(n):
            if is_valid(row, col, queens):
                queens[row] = col
                place_queen(row + 1)
                queens[row] = -1

    place_queen(0)
    return solutions
print(solve_n_queens_multiple(n=15))



```
# Tiene formato de código
```

# **03 MIN-CONFLICTS**
El algoritmo Min-Conflicts **(Mínimos Conflictos)** es un método de búsqueda local heurística. A diferencia de los algoritmos tradicionales que intentan construir una solución paso a paso desde cero **(como el backtracking)**, este algoritmo empieza con una solución completa pero incorrecta y trata de **"repararla"** iterativamente.

## *Funcionamiento en el Problema de las N-Reinas*

En el contexto de las N-Reinas, el objetivo es colocar **N** reinas en un tablero de **dimension N** sin que ninguna se ataque entre sí:
#### *Inicialización:*
Se colocan las **N** reinas en el tablero de forma aleatoria (una por columna). En este punto, habrá muchos conflictos (ataques).
#### *Selección:*
Se elige aleatoriamente una reina que esté siendo atacada actualmente (una que tenga "conflictos").
#### *Movimiento (La Heurística):*
Se mueve esa reina a la casilla donde menos ataques reciba (mínimos conflictos).
#### *Repetición:*
Se repiten estos pasos hasta que ninguna reina se ataque o se alcance un límite de intentos.

In [ ]:
# Creamos una matriz de ceros (casillas blancas)
tablero = np.zeros((n, n))

# Alternamos los 1 (casillas negras) usando 'slicing' de numpy
# Filas impares, columnas pares = 1
tablero[1::2, ::2] = 1
# Filas pares, columnas impares = 1
tablero[::2, 1::2] = 1

# Lo dibujamos usando un mapa de color binario (blanco/negro)
ax.imshow(tablero, cmap='binary', origin='upper')

# **04 REPRESENTACIÓN GRÁFICA**
Paso 1: Crear el Lienzo (La Matriz Matemática)
"Lo primero que necesitamos entender es que el ordenador no ve 'blancos y negros', ve números. Para crear el tablero, generamos una matriz de ceros y unos."

El Concepto: Usamos numpy para crear una cuadrícula donde los 0 representan las casillas blancas y los 1 las negras.


In [ ]:
# Creamos una matriz de ceros (casillas blancas)
tablero = np.zeros((n, n))

# Alternamos los 1 (casillas negras) usando 'slicing' de numpy
# Filas impares, columnas pares = 1
tablero[1::2, ::2] = 1
# Filas pares, columnas impares = 1
tablero[::2, 1::2] = 1

# Lo dibujamos usando un mapa de color binario (blanco/negro)
ax.imshow(tablero, cmap='binary', origin='upper')

NameError: name 'np' is not defined

Paso 2: Interpretar la Solución
El algoritmo usado da un vector solución [1, 3, 0, 2]. ¿Qué significa esto? Significa que:

En la Fila 0, la reina va en la Columna 1.

En la Fila 1, la reina va en la Columna 3.

Y así sucesivamente..."

Paso 3: Colocar las Piezas (El Toque Estético)

Para dibujar las reinas, no usamos una imagen externa, sino texto. Usamos el carácter Unicode de la reina ('♛') como si fuera una letra gigante.

Usamos path_effects para darle un borde negro a la reina dorada. Así resalta sobre cualquier fondo.

In [ ]:
# Recorremos la solución: el índice es la fila, el valor es la columna
for fila, columna in enumerate(solucion):
    ax.text(columna, fila, '♛',
            fontsize=40,
            color='gold', # Color dorado
            # Esto crea el borde negro alrededor de la letra
            path_effects=[path_effects.withStroke(linewidth=2, foreground='black')])

Al combinar la matriz base con nuestras coordenadas de texto, obtenemos la visualización final que confirma que nuestro algoritmo funciona.

## *El código:*

In [ ]:
import random
import matplotlib.pyplot as plt
import numpy as np
import time

#Inicializa un tablero aleatorio y ejecuta el bucle del algoritmo Min-Conflicts para encontrar una solución donde ninguna reina se ataque.
def solve_nqueens_min_conflicts(n, max_iterations=5000):
    if n <= 3 and n != 1:
        return None
    board = []
    for _ in range(n):
        fila_aleatoria = random.randint(0, n - 1)
        board.append(fila_aleatoria)


    #Calcula cuántos ataques tendría una reina en una posición específica verificando la misma fila y las diagonales.
    def count_conflicts(test_board, col, row):
        conflicts = 0
        for c in range(n):
            if c == col:
                continue
            r = test_board[c]
            if r == row:
                conflicts = conflicts + 1
            elif abs(r - row) == abs(c - col):
                conflicts = conflicts + 1
        return conflicts

    #Identifica qué columnas tienen reinas en conflicto actualmente y selecciona una al azar para intentar moverla; devuelve -1 si no hay conflictos (solución encontrada).
    def get_conflicted_column(current_board):
        conflicted_cols = []
        for c in range(n):
            numero_de_conflictos = count_conflicts(current_board, c, current_board[c])
            if numero_de_conflictos > 0:
                conflicted_cols.append(c)
        if not conflicted_cols:
            return -1
        return random.choice(conflicted_cols)

    for i in range(max_iterations):
        col = get_conflicted_column(board)
        if col == -1:
            print(f"Solución encontrada en {i} iteraciones.")
            return format_board(board, n)
        current_row = board[col]
        min_conflicts = float('inf')
        best_rows = []

        for r in range(n):
            conflicts = count_conflicts(board, col, r)
            if conflicts < min_conflicts:
                min_conflicts = conflicts
                best_rows = [r]
            elif conflicts == min_conflicts:
                best_rows.append(r)

        board[col] = random.choice(best_rows)

    print(f"Límite de {max_iterations} iteraciones alcanzado sin solución.")
    return None

#Transforma la lista numérica (donde el índice es columna y el valor es fila) en una representación visual de cadenas con puntos y 'Q'.
def format_board(board, n):
    formatted_board = []
    for row_index in board:
        row_string = '.' * row_index + 'Q' + '.' * (n - 1 - row_index)
        formatted_board.append(row_string)
    return formatted_board



#Transforma la representación visual de cadenas de puntos y 'Q' en un tablero con '♛'
def visualizar_solucion_nqueens(solucion_formateada):
    if not solucion_formateada:
        print("No se proporcionó una solución válida para graficar.")
        return
    n = len(solucion_formateada)

    #Crea el tablero
    tablero = np.zeros((n, n))
    tablero[1::2, ::2] = 1
    tablero[::2, 1::2] = 1

    #Ajusta el tamaño del tablero
    figsize_val = min(10, n)
    fig, ax = plt.subplots(figsize=(figsize_val, figsize_val))

    #Dibuja el fondo
    ax.imshow(tablero, cmap='gray_r', interpolation='nearest', origin='upper', extent=[0, n, n, 0])

    #Dibuja las Reinas
    simbolo_reina = '♛'

    fsize = max(5, 500 // n)

    for fila, valor_fila in enumerate(solucion_formateada):
        columna = valor_fila.find('Q')
        if columna != -1:
            ax.text(columna + 0.5, fila + 0.5, simbolo_reina, fontsize=fsize, ha='center', va='center', color='gold', fontweight='bold')

    ax.set_title(f"Solución para N={n}", fontsize=14)

    if n > 20:
        ax.axis('off')
    else:
        ax.set_xticks(np.arange(n))
        ax.set_yticks(np.arange(n))
        ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)

    plt.tight_layout()
    plt.show()



inicio1=time.perf_counter()
N = 30
solution = solve_nqueens_min_conflicts(N)
fin1=time.perf_counter()
inicio2=time.perf_counter()
if solution:
    print(f"Tablero de la Solución para N={N} en {fin1-inicio1} segundos:")
    for row in solution:
        print(row)
    visualizar_solucion_nqueens(solution)
    fin2=time.perf_counter()
    print(f"El tablero tardó {fin2-inicio2} segundos en generarse")
else:
    print(f"Fallo al encontrar solución para N = {N} dentro del límite de iteraciones, tardó {fin1-inicio1} segundos.")

La gran fortaleza de Min-Conflicts reside en su velocidad y escalabilidad, permitiéndole resolver problemas masivos **(incluso de un millón de reinas)** en segundos con un uso mínimo de memoria, algo imposible para algoritmos exactos. Sin embargo, su contraparte es que no garantiza una solución **(no es completo)**; al depender del azar y de arreglos locales, corre el riesgo de quedarse atascado en configuraciones sin salida **("mínimos locales")**. Además, su eficiencia destaca únicamente si el objetivo es hallar una solución, al contrario que otros métodos tradicionales **(como backtracking)**.

# **05 APLICACIONES REALES**

El problema de las N-Reinas, aunque lúdico, es un modelo de referencia para problemas del mundo real que involucran la **colocación óptima de recursos bajo ciertas restricciones**.

### Modelado de Problemas Reales

* **Benchmarking y Pruebas de IA:** Las N-Reinas son el banco de pruebas más famoso para cualquier algoritmo nuevo de búsqueda con restricciones (como los que utilizaste: Backtracking y Min-Conflicts). Demostrar que un algoritmo puede resolver el problema de forma eficiente, especialmente para $N$ grandes, prueba su valía.
* **Programación con Restricciones (CP):** Sirve como modelo para el desarrollo de solucionadores que gestionan logística compleja, como la asignación de horarios (evitando conflictos de tiempo) o la planificación de tareas en robots.
* **Problemas de Cobertura y Colocación Óptima:** Modela escenarios donde se busca colocar recursos (como antenas de telefonía, cámaras de seguridad o sensores) en un área determinada:
    * **Columna/Fila:** Evitar colocar dos recursos en la misma ubicación.
    * **Diagonal:** Evitar zonas de interferencia o redundancia entre sensores.

### El Impacto del Enfoque del Cuaderno

* **Backtracking (Método Exhaustivo):** Sus aplicaciones son comunes en sistemas donde se requiere garantizar una solución o encontrar todas ellas (e.g., puzzles, análisis de seguridad de algoritmos).
* **Min-Conflicts (Método Heurístico):** Este enfoque es crucial para problemas de gran escala donde el tiempo es crítico (por ejemplo, asignar millones de tareas o recursos en un datacenter). No garantiza una solución, pero su velocidad es inigualable en la práctica.


# **06 VISUALIZACIÓN Y CONCLUSIONES**

*   Elemento de lista
*   Elemento de lista



La visualización gráfica es el paso final que verifica la solución y nos permite entender el *trade-off* fundamental entre los dos métodos de búsqueda que hemos implementado.

### Visualización y Verificación

El código de visualización utiliza Matplotlib para **mapear la solución numérica** (el vector de columnas) a un **tablero de ajedrez** y colocar el carácter Unicode de la reina ('♛'). Esto nos permite:

1.  **Verificación Visual:** Confirmar que la solución hallada por el algoritmo es **válida**, es decir, que ninguna de las $N$ reinas se ataca.
2.  **Entendimiento de la Solución:** Mapear el vector solución $\vec{p} = [c_0, c_1, \dots, c_{N-1}]$ (donde el valor es la columna y la posición es la fila) a una representación espacial.

### Conclusiones sobre la Eficiencia

El análisis de los algoritmos de las N-Reinas ilustra el compromiso fundamental en la informática entre la **certeza** y la **velocidad**.

| Algoritmo | Énfasis | Fortalezas | Limitaciones Clave |
| :--- | :--- | :--- | :--- |
| **Backtracking** (Clásico) | **Garantía y Exhaustividad** | **Completo:** Garantiza encontrar **TODAS** las soluciones posibles. | **Escalabilidad Pobre:** Tiempo exponencial, solo práctico para $N$ pequeños ($N \le 20$). |
| **Min-Conflicts** (Heurístico) | **Velocidad y Escalabilidad** | **Extremadamente Rápido:** Puede resolver tableros muy grandes ($N \gg 100$) en segundos. | **Incompleto:** No garantiza una solución (riesgo de quedar "atascado" en mínimos locales). |

**Conclusión Final:**

La elección del algoritmo depende del objetivo:

* Si la meta es el **análisis teórico** o encontrar **todas las soluciones posibles**, se requiere el método **exhaustivo y completo** de **Backtracking**.
* Si la meta es la **aplicación práctica** en un sistema real que demanda una **solución única y rápida** (sin importar la exhaustividad), el método **heurístico** de **Min-Conflicts** es superior debido a su velocidad y escalabilidad.

